In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
#Machine Learning Models
from xgboost import XGBClassifier, plot_importance
from lightgbm import LGBMClassifier, plot_importance
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import time

from collections import Counter

In [2]:
#URL
df1 = pd.read_csv('url_features.csv').sort_values(by='url')
# df1 = pd.get_dummies(df1, columns=['tld_type'], prefix='tld')
# X1 = df1.drop(['url','label'], axis=1)
# y1 = df1['label']

In [3]:
#HTML
df2 = pd.read_csv('new_html_features.csv').sort_values(by='url')
# X2 = df2.drop(['label', 'url'], axis=1)
# y2 = df2['label']

In [4]:
#DOM
df3 = pd.read_csv('dom_features_output.csv').sort_values(by='url')
# X3 = df3.drop(['URL','Labeled'], axis=1)
# y3 = df3['Labeled']

In [5]:
df4 = pd.merge(df1, df2, on="url")
df5 =pd.merge(df3, df4, on="url")

#Check to make sure the two labels are identical
# (df3['label_x'] == df3['label_y']).all()

# for col in df5.columns:
#     print(col)

# df4 = df4.rename(columns={'label_x': 'label'})

df5 = pd.get_dummies(df5, columns=['tld_type'], prefix='tld')
X4 = df5.drop(['url','label', 'label_x', 'label_y'], axis=1)
y4 = df5['label']



# print(y3)

url
label
dom_feat_0
dom_feat_1
dom_feat_2
dom_feat_3
dom_feat_4
dom_feat_5
dom_feat_6
dom_feat_7
dom_feat_8
dom_feat_9
dom_feat_10
dom_feat_11
dom_feat_12
dom_feat_13
dom_feat_14
dom_feat_15
dom_feat_16
dom_feat_17
dom_feat_18
dom_feat_19
dom_feat_20
dom_feat_21
dom_feat_22
dom_feat_23
dom_feat_24
dom_feat_25
dom_feat_26
dom_feat_27
dom_feat_28
dom_feat_29
dom_feat_30
dom_feat_31
dom_feat_32
dom_feat_33
dom_feat_34
dom_feat_35
dom_feat_36
dom_feat_37
dom_feat_38
dom_feat_39
dom_feat_40
dom_feat_41
dom_feat_42
dom_feat_43
dom_feat_44
dom_feat_45
dom_feat_46
dom_feat_47
dom_feat_48
dom_feat_49
dom_feat_50
dom_feat_51
dom_feat_52
dom_feat_53
dom_feat_54
dom_feat_55
dom_feat_56
dom_feat_57
dom_feat_58
dom_feat_59
dom_feat_60
dom_feat_61
dom_feat_62
dom_feat_63
dom_feat_64
dom_feat_65
dom_feat_66
dom_feat_67
dom_feat_68
dom_feat_69
dom_feat_70
dom_feat_71
dom_feat_72
dom_feat_73
dom_feat_74
dom_feat_75
dom_feat_76
dom_feat_77
dom_feat_78
dom_feat_79
dom_feat_80
dom_feat_81
dom_feat_82
dom_

In [6]:
# #Combined
# df4 = pd.read_csv('combined_features.csv').sort_values(by='URL')
# df4 = pd.get_dummies(df4, columns=['tld_type'], prefix='tld')
# X4 = df4.drop(['URL','Labeled'], axis=1)
# y4 = df4['Labeled']

In [7]:
X_train4, X_test4, y_train4, y_test4 = train_test_split(X4, y4, stratify=y4, test_size=0.2, random_state=42)

In [8]:
models = {
    "XGBoost": XGBClassifier(random_state=42),
    "LightGBM": LGBMClassifier(random_state=42),
    "Random Forest": RandomForestClassifier( random_state=42),
    "K-Nearest Neighbor": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(random_state=42)
}

In [9]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def run_cv_block(X_train, y_train, X_test, y_test, models, dataset_name):
    print(f"\n\n===== {dataset_name} =====")
    cv_results = {}

    for name, model in models.items():
        print(f"\n{name} - 5-Fold CV on 80% Training Set")

        fold_accuracies = []
        fold_precisions = []
        fold_recalls = []
        fold_f1s = []
        fold_reports = []

        training_times = []
        prediction_times = []

        for train_idx, val_idx in kfold.split(X_train, y_train):
            X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

            start_train = time.time()
            model.fit(X_train_fold, y_train_fold)
            end_train = time.time()

            start_pred = time.time()
            y_val_pred = model.predict(X_val_fold)
            end_pred = time.time()

            fold_accuracies.append(accuracy_score(y_val_fold, y_val_pred))
            fold_precisions.append(precision_score(y_val_fold, y_val_pred, average='weighted', zero_division=0))
            fold_recalls.append(recall_score(y_val_fold, y_val_pred, average='weighted', zero_division=0))
            fold_f1s.append(f1_score(y_val_fold, y_val_pred, average='weighted', zero_division=0))

            report = classification_report(y_val_fold, y_val_pred, output_dict=True, zero_division=0)
            fold_reports.append(pd.DataFrame(report).transpose())

            training_times.append(end_train - start_train)
            prediction_times.append(end_pred - start_pred)

        avg_report_df = pd.concat(fold_reports).groupby(level=0).mean()

        model.fit(X_train, y_train)
        y_test_pred = model.predict(X_test)

        test_accuracy = accuracy_score(y_test, y_test_pred)
        test_conf_matrix = confusion_matrix(y_test, y_test_pred)
        test_class_report = classification_report(y_test, y_test_pred)

        # Store results
        cv_results[name] = {
            "cv_avg_accuracy": np.mean(fold_accuracies),
            "cv_avg_precision": np.mean(fold_precisions),
            "cv_avg_recall": np.mean(fold_recalls),
            "cv_avg_f1": np.mean(fold_f1s),
            "cv_classification_report_df": avg_report_df,
            "avg_training_time": np.mean(training_times),
            "avg_prediction_time": np.mean(prediction_times),
            "test_accuracy": test_accuracy,
            "test_conf_matrix": test_conf_matrix,
            "test_class_report": test_class_report,
        }

    for name, result in cv_results.items():
        print(f"\n{'='*70}\n{name} | DATASET: {dataset_name}")
        print(f"CV Average Accuracy     : {result['cv_avg_accuracy']:.4f}")
        print(f"CV Avg Precision        : {result['cv_avg_precision']:.4f}")
        print(f"CV Avg Recall           : {result['cv_avg_recall']:.4f}")
        print(f"CV Avg F1-Score         : {result['cv_avg_f1']:.4f}")
        print(f"Average Training Time   : {result['avg_training_time']:.4f} s")
        print(f"Average Prediction Time : {result['avg_prediction_time']:.4f} s")
        
        print("\nAverage Classification Report from 5-Fold CV:")
        print(result["cv_classification_report_df"])

        print(f"\nTest Set Accuracy (20%) : {result['test_accuracy']:.4f}")
        print("Test Set Confusion Matrix:")
        print(result["test_conf_matrix"])
        print("Test Set Classification Report:")
        print(result["test_class_report"])

    return cv_results

results_combined = run_cv_block(X_train4, y_train4, X_test4, y_test4, models, "URL + HTML")




===== URL + HTML =====

XGBoost - 5-Fold CV on 80% Training Set

LightGBM - 5-Fold CV on 80% Training Set
[LightGBM] [Info] Number of positive: 13511, number of negative: 432613
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.556777 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 83292
[LightGBM] [Info] Number of data points in the train set: 446124, number of used features: 588
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.030285 -> initscore=-3.466339
[LightGBM] [Info] Start training from score -3.466339
[LightGBM] [Info] Number of positive: 13512, number of negative: 432613
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.609510 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 83324
[LightGBM] [Info] Number of data points in the train set: 446125, number of used features: 590
[LightGBM] [Info] [binary:Boos